In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON

In [2]:
# %% Load dataset
sampling_rate = 22050
method_algo = 'mfcc'

inputs, targets, mapping = Directory.load_json_data(f'processed/{method_algo}/{method_algo}_{sampling_rate}.json',
                                         inputs_fieldname=method_algo)

In [3]:
# SPLIT DOS DADOS
random_state = 42

inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          shuffle=True,
                                                                          random_state=random_state)

In [4]:
def create_dense_model():
    # build the network architecture
    model = keras.Sequential([
        # input layer
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),

        # 1st hidden layer
        keras.layers.Dense(512, activation='relu'),

        # 2nd hidden layer
        keras.layers.Dense(256, activation='relu'),

        # 3rd hidden layer
        keras.layers.Dense(128, activation='relu'),

        # output layer
        keras.layers.Dense(len(mapping), activation='softmax'),
    ])

    return model

In [5]:
# CRIA O MODELO
model = create_dense_model()

# COMPILA A REDE
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

In [6]:
# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/deep/{timestamp}')

JSON.create_json_file(f'models/deep/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/deep/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [7]:
# TREINA O MODELO
history = model.fit(inputs_train, targets_train,
            validation_data=(inputs_test, targets_test),
            epochs=10000,
            batch_size=128,
            callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/10000
111/111 [==============================] - 2s 14ms/step - loss: 179.4568 - accuracy: 0.0139 - val_loss: 4.9163 - val_accuracy: 0.0107
Epoch 2/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.8281 - accuracy: 0.0126 - val_loss: 4.7562 - val_accuracy: 0.0110
Epoch 3/10000
111/111 [==============================] - 1s 11ms/step - loss: 4.7106 - accuracy: 0.0112 - val_loss: 4.7379 - val_accuracy: 0.0110
Epoch 4/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.6866 - accuracy: 0.0137 - val_loss: 4.7337 - val_accuracy: 0.0110
Epoch 5/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.6807 - accuracy: 0.0121 - val_loss: 4.7375 - val_accuracy: 0.0127
Epoch 6/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.6796 - accuracy: 0.0115 - val_loss: 4.7351 - val_accuracy: 0.0127
Epoch 7/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.6760 - accuracy: 0.0135 - val_loss: 4.7392

Epoch 57/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.6012 - accuracy: 0.0278 - val_loss: 4.6744 - val_accuracy: 0.0234
Epoch 58/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.6102 - accuracy: 0.0271 - val_loss: 4.6579 - val_accuracy: 0.0206
Epoch 59/10000
111/111 [==============================] - 1s 11ms/step - loss: 4.6134 - accuracy: 0.0257 - val_loss: 4.7250 - val_accuracy: 0.0215
Epoch 60/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.5931 - accuracy: 0.0318 - val_loss: 4.7066 - val_accuracy: 0.0220
Epoch 61/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.5915 - accuracy: 0.0301 - val_loss: 4.6871 - val_accuracy: 0.0229
Epoch 62/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.6030 - accuracy: 0.0287 - val_loss: 4.7280 - val_accuracy: 0.0237
Epoch 63/10000
111/111 [==============================] - 1s 12ms/step - loss: 4.5934 - accuracy: 0.0303 - val_loss: 4

111/111 [==============================] - 1s 12ms/step - loss: 3.9513 - accuracy: 0.1275 - val_loss: 4.2678 - val_accuracy: 0.1044
Epoch 113/10000
111/111 [==============================] - 1s 12ms/step - loss: 3.9577 - accuracy: 0.1258 - val_loss: 4.6392 - val_accuracy: 0.1095
Epoch 114/10000
111/111 [==============================] - 1s 11ms/step - loss: 3.9415 - accuracy: 0.1339 - val_loss: 4.7260 - val_accuracy: 0.1160
Epoch 115/10000
111/111 [==============================] - 1s 11ms/step - loss: 3.9162 - accuracy: 0.1317 - val_loss: 4.4355 - val_accuracy: 0.1163
Epoch 116/10000
111/111 [==============================] - 1s 11ms/step - loss: 3.8859 - accuracy: 0.1414 - val_loss: 4.2831 - val_accuracy: 0.1242
Epoch 117/10000
111/111 [==============================] - 1s 11ms/step - loss: 3.8537 - accuracy: 0.1393 - val_loss: 4.4911 - val_accuracy: 0.1222
Epoch 118/10000
111/111 [==============================] - 1s 12ms/step - loss: 3.8385 - accuracy: 0.1481 - val_loss: 4.7251 - v

111/111 [==============================] - 1s 12ms/step - loss: 2.5619 - accuracy: 0.3935 - val_loss: 3.8486 - val_accuracy: 0.3156
Epoch 168/10000
111/111 [==============================] - 1s 12ms/step - loss: 2.5176 - accuracy: 0.3974 - val_loss: 4.1856 - val_accuracy: 0.3376
Epoch 169/10000
111/111 [==============================] - 1s 12ms/step - loss: 2.5241 - accuracy: 0.3985 - val_loss: 3.9774 - val_accuracy: 0.3254
Epoch 170/10000
111/111 [==============================] - 1s 12ms/step - loss: 2.4908 - accuracy: 0.4025 - val_loss: 4.2049 - val_accuracy: 0.3345
Epoch 171/10000
111/111 [==============================] - 1s 12ms/step - loss: 2.4274 - accuracy: 0.4130 - val_loss: 4.0753 - val_accuracy: 0.3410
Epoch 172/10000
111/111 [==============================] - 1s 12ms/step - loss: 2.4095 - accuracy: 0.4125 - val_loss: 4.1588 - val_accuracy: 0.3489
Epoch 173/10000
111/111 [==============================] - 1s 12ms/step - loss: 2.3973 - accuracy: 0.4182 - val_loss: 3.8583 - v

111/111 [==============================] - 1s 12ms/step - loss: 1.0403 - accuracy: 0.7300 - val_loss: 3.3008 - val_accuracy: 0.5905
Epoch 223/10000
111/111 [==============================] - 1s 12ms/step - loss: 1.0011 - accuracy: 0.7422 - val_loss: 3.4711 - val_accuracy: 0.6111
Epoch 224/10000
111/111 [==============================] - 1s 12ms/step - loss: 1.0238 - accuracy: 0.7434 - val_loss: 3.3018 - val_accuracy: 0.6051
Epoch 225/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.9766 - accuracy: 0.7527 - val_loss: 3.2108 - val_accuracy: 0.6142
Epoch 226/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.9358 - accuracy: 0.7575 - val_loss: 3.2025 - val_accuracy: 0.6037
Epoch 227/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.9067 - accuracy: 0.7727 - val_loss: 3.2566 - val_accuracy: 0.6173
Epoch 228/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.8587 - accuracy: 0.7795 - val_loss: 3.2769 - v

...

111/111 [==============================] - 1s 12ms/step - loss: 0.0586 - accuracy: 0.9865 - val_loss: 2.7271 - val_accuracy: 0.8273
Epoch 333/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0590 - accuracy: 0.9861 - val_loss: 2.8456 - val_accuracy: 0.8273
Epoch 334/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0540 - accuracy: 0.9868 - val_loss: 2.6677 - val_accuracy: 0.8332
Epoch 335/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0431 - accuracy: 0.9878 - val_loss: 2.7259 - val_accuracy: 0.8202
Epoch 336/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0769 - accuracy: 0.9851 - val_loss: 2.5925 - val_accuracy: 0.8202
Epoch 337/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0603 - accuracy: 0.9841 - val_loss: 2.7669 - val_accuracy: 0.8242
Epoch 338/10000
111/111 [==============================] - 1s 11ms/step - loss: 0.0478 - accuracy: 0.9880 - val_loss: 2.7433 - v

111/111 [==============================] - 1s 12ms/step - loss: 0.0465 - accuracy: 0.9924 - val_loss: 2.8099 - val_accuracy: 0.8490
Epoch 388/10000
111/111 [==============================] - 1s 11ms/step - loss: 0.0260 - accuracy: 0.9930 - val_loss: 3.0872 - val_accuracy: 0.8467
Epoch 389/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0534 - accuracy: 0.9898 - val_loss: 2.9249 - val_accuracy: 0.8394
Epoch 390/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0371 - accuracy: 0.9909 - val_loss: 2.9611 - val_accuracy: 0.8456
Epoch 391/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0443 - accuracy: 0.9906 - val_loss: 2.9055 - val_accuracy: 0.8561
Epoch 392/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0181 - accuracy: 0.9948 - val_loss: 2.8508 - val_accuracy: 0.8549
Epoch 393/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0300 - accuracy: 0.9928 - val_loss: 2.7584 - v

111/111 [==============================] - 1s 12ms/step - loss: 0.0109 - accuracy: 0.9970 - val_loss: 2.7466 - val_accuracy: 0.8693
Epoch 443/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0118 - accuracy: 0.9968 - val_loss: 2.9155 - val_accuracy: 0.8671
Epoch 444/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0195 - accuracy: 0.9959 - val_loss: 2.9478 - val_accuracy: 0.8654
Epoch 445/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0170 - accuracy: 0.9959 - val_loss: 2.7570 - val_accuracy: 0.8744
Epoch 446/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0170 - accuracy: 0.9962 - val_loss: 2.8925 - val_accuracy: 0.8727
Epoch 447/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0158 - accuracy: 0.9969 - val_loss: 3.0493 - val_accuracy: 0.8600
Epoch 448/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0149 - accuracy: 0.9958 - val_loss: 2.8224 - v

111/111 [==============================] - 2s 15ms/step - loss: 0.0292 - accuracy: 0.9942 - val_loss: 3.3080 - val_accuracy: 0.8685
Epoch 498/10000
111/111 [==============================] - 2s 20ms/step - loss: 0.0171 - accuracy: 0.9957 - val_loss: 3.5492 - val_accuracy: 0.8688
Epoch 499/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0220 - accuracy: 0.9959 - val_loss: 3.4529 - val_accuracy: 0.8577
Epoch 500/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0232 - accuracy: 0.9957 - val_loss: 3.3150 - val_accuracy: 0.8707
Epoch 501/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0143 - accuracy: 0.9966 - val_loss: 2.8201 - val_accuracy: 0.8806
Epoch 502/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0085 - accuracy: 0.9980 - val_loss: 2.8830 - val_accuracy: 0.8829
Epoch 503/10000
111/111 [==============================] - 1s 12ms/step - loss: 0.0122 - accuracy: 0.9973 - val_loss: 2.9448 - v

111/111 [==============================] - 2s 22ms/step - loss: 0.0298 - accuracy: 0.9956 - val_loss: 3.2336 - val_accuracy: 0.8817
Epoch 553/10000
111/111 [==============================] - 2s 22ms/step - loss: 0.0130 - accuracy: 0.9971 - val_loss: 3.3651 - val_accuracy: 0.8784
Epoch 554/10000
111/111 [==============================] - 2s 21ms/step - loss: 0.0109 - accuracy: 0.9976 - val_loss: 3.4280 - val_accuracy: 0.8795
Epoch 555/10000
111/111 [==============================] - 2s 22ms/step - loss: 0.0057 - accuracy: 0.9985 - val_loss: 3.3604 - val_accuracy: 0.8800
Epoch 556/10000
111/111 [==============================] - 2s 22ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 3.3542 - val_accuracy: 0.8792
Epoch 557/10000
111/111 [==============================] - 3s 23ms/step - loss: 0.0057 - accuracy: 0.9985 - val_loss: 3.3408 - val_accuracy: 0.8803
Epoch 558/10000
111/111 [==============================] - 3s 23ms/step - loss: 0.0052 - accuracy: 0.9985 - val_loss: 3.3342 - v

In [8]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/deep/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/deep/{timestamp}/graph_loss.png')
plt.close()

In [9]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

    
[_,score_test]= model.evaluate(inputs_test, targets_test, batch_size=128)

[_,score_train] = model.evaluate(inputs_train, targets_train, batch_size=128)

targets_hat = np.argmax(model.predict(inputs_test), axis=-1)

dump_info = {
    'method': 'Deep Learning',
    'seed': random_state,
    'feature_method': method_algo,
    'sample_rate': sampling_rate,
    'train_test': [len(inputs_train), len(inputs_test)],
    'score_train': score_train,
    'score_test': score_test,
    'f1_micro': f1_score(targets_hat, targets_test, average='micro'),
    'f1_macro': f1_score(targets_hat, targets_test, average='macro'),
}

JSON.create_json_file(f'models/deep/{timestamp}/info.json', dump_info)

111/111 [==============================] - 1s 8ms/step - loss: 0.0971 - accuracy: 0.9814


In [10]:
# RENOMEIA A PASTA
Directory.create_directory(f'models/deep/{method_algo}_{sampling_rate}')
Directory.rename_directory(f'models/deep/{timestamp}',
                   f'models/deep/{method_algo}_{sampling_rate}/acc{str(int(score_test * 10000)).zfill(4)}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')